## This file transforms and cleans the GDP data. Significant changes had to be made to deal with the Virginia FIPS code anamolies, that don't occur in other data sources

Virginia GDP data is odd as it combines multiple some fips codes together, particularly townships within a larger county area. (Virginia labels cities as their own FIPS). Subsequently, we have to divide out GDP using the number of workers in a FIPS code. 

In [1]:
import pandas as pd

In [2]:
gdp_df = pd.read_csv('C:/Users/zapate/Desktop/YEAR 2/GDP/CAGDP1__ALL_AREAS_2001_2021.csv', encoding='latin-1')
gdp_df = gdp_df[gdp_df['Description'] == 'Real GDP (thousands of chained 2012 dollars) ']
gdp_df['GeoFIPS'] = gdp_df['GeoFIPS'].str.replace('"', '')
gdp_df['GeoFIPS'] =gdp_df['GeoFIPS'].str.strip()
gdp_df = gdp_df.drop(columns = ['GeoName', 'Region', 'TableName', 'LineCode',
       'IndustryClassification', 'Description', 'Unit'])

gdp_df_melt = pd.melt(gdp_df, id_vars = ['GeoFIPS'], value_vars = ['2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'] ).rename(columns = {'value':'GDP'})

gdp_df_melt['State_Code'] = gdp_df_melt['GeoFIPS'].astype(str).str[0:2]
gdp_df_melt['County_Code'] = gdp_df_melt['GeoFIPS'].astype(str).str[2:]

gdp_df_melt = gdp_df_melt[gdp_df_melt['County_Code'] != '000']

gdp_df_melt

,GeoFIPS,variable,GDP,State_Code,County_Code
2,01001,2001,949800,01,001
3,01003,2001,4007706,01,003
4,01005,2001,812751,01,005
5,01007,2001,292495,01,007
6,01009,2001,810054,01,009
...,...,...,...,...,...
66725,56037,2021,3125840,56,037
66726,56039,2021,2862327,56,039
66727,56041,2021,771851,56,041
66728,56043,2021,346859,56,043


In [3]:
#dealing with virginia
virginia = pd.read_csv('Virginia_conversions.csv')
virginia = pd.concat([virginia[['BEA_FIPS', 'FIPS1']],
                      virginia[['BEA_FIPS', 'FIPS2']].rename(columns={'FIPS2':'FIPS1'}),
                      virginia[['BEA_FIPS', 'FIPS3']].rename(columns={'FIPS3':'FIPS1'})]).dropna().rename(columns = {'FIPS1':'True_Fips'})

virginia['True_Fips'] = virginia['True_Fips'].astype(int).astype(str)
virginia['BEA_FIPS'] = virginia['BEA_FIPS'].astype(int).astype(str)

virginia.head(), virginia.dtypes

(  BEA_FIPS True_Fips
 0    51901     51003
 1    51903     51005
 2    51907     51015
 3    51911     51031
 4    51913     51035,
 BEA_FIPS     object
 True_Fips    object
 dtype: object)

In [4]:
virginia['BEA_State'] = virginia['BEA_FIPS'].str[0:2]
virginia['BEA_County'] = virginia['BEA_FIPS'].str[2:]
virginia['True_State'] = virginia['True_Fips'].str[0:2]
virginia['True_County'] = virginia['True_Fips'].str[2:]

In [6]:
emp_df = pd.read_csv('Final_Unemployment_County.csv')
emp_df = emp_df[['State_Code', 'County_Code', 'County_name',
       'Year','Labor_Force', 'Employed', 'Unemployed',
       'Unemployment_Rate']].dropna(subset = ['State_Code', 'County_Code','Year'])

emp_df['State_Code'] = emp_df['State_Code'].astype(int).astype(str)
emp_df['County_Code'] = emp_df['County_Code'].astype(int).astype(str)
emp_df['Year'] = emp_df['Year'].astype(int).astype(str)
emp_df

,State_Code,County_Code,County_name,Year,Labor_Force,Employed,Unemployed,Unemployment_Rate
0,1,1,"Autauga County, AL",2000,21861,20971,890,4.1
1,1,3,"Baldwin County, AL",2000,69979,67370,2609,3.7
2,1,5,"Barbour County, AL",2000,11449,10812,637,5.6
3,1,7,"Bibb County, AL",2000,8623,8160,463,5.4
4,1,9,"Blount County, AL",2000,25266,24375,891,3.5
...,...,...,...,...,...,...,...,...
70858,72,145,"Vega Baja Municipio, PR",2021,14172,12748,1424,10
70859,72,147,"Vieques Municipio, PR",2021,2804,2469,335,11.9
70860,72,149,"Villalba Municipio, PR",2021,7697,6796,901,11.7
70861,72,151,"Yabucoa Municipio, PR",2021,9156,8216,940,10.3


In [19]:
emp_df.dtypes, virginia.dtypes

(State_Code           object
 County_Code          object
 County_name          object
 Year                 object
 Labor_Force          object
 Employed             object
 Unemployed           object
 Unemployment_Rate    object
 dtype: object,
 BEA_FIPS       object
 True_Fips      object
 BEA_State      object
 BEA_County     object
 True_State     object
 True_County    object
 dtype: object)

In [21]:
emp_df['County_Code'] = emp_df['County_Code'].astype(int)
virginia['True_County'] = virginia['True_County'].astype(int)


emp_ratio = emp_df.merge(virginia, how= 'left',
                         left_on = ['State_Code','County_Code'], right_on = ['True_State', 'True_County'])

emp_ratio = emp_ratio.dropna(subset=['True_Fips'])
emp_ratio

,State_Code,County_Code,County_name,Year,Labor_Force,Employed,Unemployed,Unemployment_Rate,BEA_FIPS,True_Fips,BEA_State,BEA_County,True_State,True_County
2818,51,3,"Albemarle County, VA",2000,43572,42829,743,1.7,51901,51003,51,901,51,3.0
2819,51,5,"Alleghany County, VA",2000,7733,7500,233,3,51903,51005,51,903,51,5.0
2824,51,15,"Augusta County, VA",2000,34604,33935,669,1.9,51907,51015,51,907,51,15.0
2832,51,31,"Campbell County, VA",2000,26513,25904,609,2.3,51911,51031,51,911,51,31.0
2834,51,35,"Carroll County, VA",2000,14818,14222,596,4,51913,51035,51,913,51,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70548,51,775,"Salem city, VA",2021,12579,12129,450,3.6,51944,51775,51,944,51,775.0
70549,51,790,"Staunton city, VA",2021,12049,11588,461,3.8,51907,51790,51,907,51,790.0
70552,51,820,"Waynesboro city, VA",2021,10471,10038,433,4.1,51907,51820,51,907,51,820.0
70553,51,830,"Williamsburg city, VA",2021,6699,6375,324,4.8,51931,51830,51,931,51,830.0


In [22]:
emp_sums = emp_ratio[['Employed', 'BEA_FIPS', 'Year']]
emp_sums['Employed'] = emp_sums['Employed'].astype(float)
emp_sums = emp_sums.groupby(['BEA_FIPS', 'Year']).sum().reset_index().sort_values('Employed', ascending=False)
emp_sums = emp_sums.rename(columns = {'Employed':'Total_Employed'})
emp_sums

C:\Users\zapate\AppData\Local\Temp\ipykernel_31092\3005573254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emp_sums['Employed'] = emp_sums['Employed'].astype(float)


,BEA_FIPS,Year,Total_Employed
151,51919,2019,643945.0
150,51919,2018,635339.0
149,51919,2017,631004.0
153,51919,2021,621757.0
148,51919,2016,618473.0
...,...,...,...
176,51923,2000,7756.0
178,51923,2002,7670.0
182,51923,2006,7508.0
181,51923,2005,7434.0


In [25]:
emp_ratio = emp_ratio.merge(emp_sums, how = 'left', left_on = ['BEA_FIPS', 'Year'], right_on = ['BEA_FIPS', 'Year'])
emp_ratio['GDP_percent'] = emp_ratio['Employed'].astype(float) / emp_ratio['Total_Employed']
emp_ratio = emp_ratio[['True_Fips', 'Year', 'GDP_percent', 'BEA_FIPS']]


In [26]:
emp_ratio[emp_ratio['True_Fips'] == '51003']

,True_Fips,Year,GDP_percent,BEA_FIPS
0,51003,2000,0.694825,51901
50,51003,2001,0.701237,51901
100,51003,2002,0.694549,51901
150,51003,2003,0.699025,51901
200,51003,2004,0.701927,51901
250,51003,2005,0.707534,51901
300,51003,2006,0.711184,51901
350,51003,2007,0.708977,51901
400,51003,2008,0.709819,51901
450,51003,2009,0.708029,51901


In [27]:
gdp_df_melt1 = gdp_df_melt.merge(emp_ratio, how='left', left_on = ['variable', 'GeoFIPS'], right_on = ['Year','BEA_FIPS'])
gdp_df_melt1

,GeoFIPS,variable,GDP,State_Code,County_Code,True_Fips,Year,GDP_percent,BEA_FIPS
0,01001,2001,949800,01,001,NaN,NaN,NaN,NaN
1,01003,2001,4007706,01,003,NaN,NaN,NaN,NaN
2,01005,2001,812751,01,005,NaN,NaN,NaN,NaN
3,01007,2001,292495,01,007,NaN,NaN,NaN,NaN
4,01009,2001,810054,01,009,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
66040,56037,2021,3125840,56,037,NaN,NaN,NaN,NaN
66041,56039,2021,2862327,56,039,NaN,NaN,NaN,NaN
66042,56041,2021,771851,56,041,NaN,NaN,NaN,NaN
66043,56043,2021,346859,56,043,NaN,NaN,NaN,NaN


In [28]:
gdp_df_melt1[gdp_df_melt1['State_Code'] == "51"]

,GeoFIPS,variable,GDP,State_Code,County_Code,True_Fips,Year,GDP_percent,BEA_FIPS
2824,51001,2001,2099713,51,001,NaN,NaN,NaN,NaN
2825,51007,2001,234853,51,007,NaN,NaN,NaN,NaN
2826,51009,2001,744189,51,009,NaN,NaN,NaN,NaN
2827,51011,2001,291071,51,011,NaN,NaN,NaN,NaN
2828,51013,2001,22868634,51,013,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
65851,51953,2021,3288442,51,953,51520,2021,0.222481,51953
65852,51955,2021,1363863,51,955,51195,2021,0.887607,51955
65853,51955,2021,1363863,51,955,51720,2021,0.112393,51955
65854,51958,2021,2497814,51,958,51199,2021,0.838249,51958


In [29]:
gdp_df_melt1['GDP_percent'] = gdp_df_melt1['GDP_percent'].fillna(1)
gdp_df_melt1['GDP'] = pd.to_numeric(gdp_df_melt1['GDP'], errors = 'coerce')
gdp_df_melt1['True_GDP'] = gdp_df_melt1['GDP'].astype(float) * gdp_df_melt1['GDP_percent']
gdp_df_melt1

,GeoFIPS,variable,GDP,State_Code,County_Code,True_Fips,Year,GDP_percent,BEA_FIPS,True_GDP
0,01001,2001,949800.0,01,001,NaN,NaN,1.0,NaN,949800.0
1,01003,2001,4007706.0,01,003,NaN,NaN,1.0,NaN,4007706.0
2,01005,2001,812751.0,01,005,NaN,NaN,1.0,NaN,812751.0
3,01007,2001,292495.0,01,007,NaN,NaN,1.0,NaN,292495.0
4,01009,2001,810054.0,01,009,NaN,NaN,1.0,NaN,810054.0
...,...,...,...,...,...,...,...,...,...,...
66040,56037,2021,3125840.0,56,037,NaN,NaN,1.0,NaN,3125840.0
66041,56039,2021,2862327.0,56,039,NaN,NaN,1.0,NaN,2862327.0
66042,56041,2021,771851.0,56,041,NaN,NaN,1.0,NaN,771851.0
66043,56043,2021,346859.0,56,043,NaN,NaN,1.0,NaN,346859.0


In [30]:
gdp_df_melt1[gdp_df_melt1['State_Code'] == '51']

,GeoFIPS,variable,GDP,State_Code,County_Code,True_Fips,Year,GDP_percent,BEA_FIPS,True_GDP
2824,51001,2001,2099713.0,51,001,NaN,NaN,1.000000,NaN,2.099713e+06
2825,51007,2001,234853.0,51,007,NaN,NaN,1.000000,NaN,2.348530e+05
2826,51009,2001,744189.0,51,009,NaN,NaN,1.000000,NaN,7.441890e+05
2827,51011,2001,291071.0,51,011,NaN,NaN,1.000000,NaN,2.910710e+05
2828,51013,2001,22868634.0,51,013,NaN,NaN,1.000000,NaN,2.286863e+07
...,...,...,...,...,...,...,...,...,...,...
65851,51953,2021,3288442.0,51,953,51520,2021,0.222481,51953,7.316174e+05
65852,51955,2021,1363863.0,51,955,51195,2021,0.887607,51955,1.210574e+06
65853,51955,2021,1363863.0,51,955,51720,2021,0.112393,51955,1.532892e+05
65854,51958,2021,2497814.0,51,958,51199,2021,0.838249,51958,2.093790e+06


In [31]:
gdp_df_melt1.True_Fips.fillna(gdp_df_melt1.GeoFIPS, inplace = True)


In [33]:
gdp_df_melt1[['variable',
              'True_GDP',
              'True_Fips']].rename(columns ={'variable':'YEAR'}).to_csv('Calculated_GDP.csv',index=False)

In [ ]:
gdp_df_melt1